In [2]:
import matplotlib.pyplot as plt
from matplotlib.image import imread
from mpl_toolkits import mplot3d
from matplotlib import gridspec
from copy import deepcopy
from mlxtend.plotting import plot_decision_regions
import seaborn as sns
import pandas as pd
from tqdm.notebook import tqdm
from scipy.special import softmax
from scipy.spatial.distance import cdist
import numpy as np
from sklearn import tree
import itertools
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.metrics import auc
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.datasets import make_classification, load_wine
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from torchvision import datasets
import pyfpgrowth
from torchvision import transforms

In [55]:
np.random.seed(0)
l = 200
x = np.vstack([np.random.choice([0,1], size=l, p=[0.5,0.5]),
              np.random.choice([0,1], size=l, p=[0.5,0.5])]).T
x = np.hstack([x,x*np.random.choice([0,1],size=[1,2],p=[0.2,0.8])])
x

array([[1, 0, 1, 0],
       [1, 1, 1, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [1, 1, 1, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 1],
       [1, 0, 1, 0],
       [0, 0, 0, 0],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 0, 1, 0],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [1, 0, 1, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [1, 0, 1, 0],
       [0, 0, 0, 0],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [1, 0, 1, 0],
       [1, 1, 1, 1],
       [0, 1, 0, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [0, 0, 0, 0],
       [1, 1, 1, 1],
       [1, 0, 1, 0],
       [0, 1, 0, 1],
       [0, 0,

In [52]:
transactions = list(map(lambda a: tuple(np.where(a == 1)[0]),x))
transactions

[(0, 2),
 (0, 1, 2, 3),
 (0, 2),
 (0, 2),
 (),
 (0, 2),
 (1, 3),
 (0, 2),
 (0, 1, 2, 3),
 (1, 3),
 (0, 1, 2, 3),
 (0, 2),
 (0, 1, 2, 3),
 (0, 2),
 (),
 (1, 3),
 (),
 (0, 2),
 (0, 1, 2, 3),
 (0, 1, 2, 3),
 (0, 1, 2, 3),
 (0, 1, 2, 3),
 (),
 (0, 1, 2, 3),
 (),
 (0, 1, 2, 3),
 (1, 3),
 (0, 2),
 (0, 1, 2, 3),
 (1, 3),
 (),
 (0, 2),
 (1, 3),
 (0, 2),
 (),
 (0, 2),
 (0, 2),
 (0, 2),
 (0, 2),
 (0, 1, 2, 3),
 (1, 3),
 (),
 (0, 1, 2, 3),
 (),
 (0, 1, 2, 3),
 (0, 2),
 (1, 3),
 (),
 (),
 (),
 (0, 2),
 (1, 3),
 (0, 1, 2, 3),
 (1, 3),
 (1, 3),
 (),
 (0, 1, 2, 3),
 (1, 3),
 (1, 3),
 (),
 (),
 (),
 (0, 2),
 (1, 3),
 (1, 3),
 (1, 3),
 (0, 2),
 (1, 3),
 (0, 2),
 (),
 (0, 1, 2, 3),
 (1, 3),
 (0, 2),
 (0, 1, 2, 3),
 (0, 2),
 (1, 3),
 (1, 3),
 (1, 3),
 (1, 3),
 (1, 3),
 (),
 (1, 3),
 (1, 3),
 (0, 2),
 (0, 2),
 (),
 (0, 1, 2, 3),
 (),
 (0, 2),
 (0, 1, 2, 3),
 (),
 (0, 2),
 (1, 3),
 (0, 2),
 (1, 3),
 (),
 (0, 2),
 (1, 3),
 (0, 1, 2, 3),
 (1, 3),
 (0, 1, 2, 3),
 (1, 3),
 (0, 2),
 (0, 2),
 (),
 (0, 2),
 (0, 2

In [53]:
patterns = pyfpgrowth.find_frequent_patterns(transactions,0)
patterns

{(1,): 95,
 (1, 2): 45,
 (0, 1): 45,
 (0, 1, 2): 45,
 (3,): 95,
 (1, 3): 95,
 (2, 3): 45,
 (0, 3): 45,
 (1, 2, 3): 45,
 (0, 1, 3): 45,
 (0, 2, 3): 45,
 (0, 1, 2, 3): 45,
 (0,): 106,
 (2,): 106,
 (0, 2): 106}

In [54]:
rules = pyfpgrowth.generate_association_rules(patterns,0)
rules

{(1,): ((0, 2, 3), 0.47368421052631576),
 (2,): ((0,), 1.0),
 (0,): ((2,), 1.0),
 (0, 1): ((2, 3), 1.0),
 (0, 2): ((1, 3), 0.42452830188679247),
 (1, 2): ((0, 3), 1.0),
 (3,): ((0, 1, 2), 0.47368421052631576),
 (1, 3): ((0, 2), 0.47368421052631576),
 (2, 3): ((0, 1), 1.0),
 (0, 3): ((1, 2), 1.0),
 (0, 1, 2): ((3,), 1.0),
 (0, 1, 3): ((2,), 1.0),
 (0, 2, 3): ((1,), 1.0),
 (1, 2, 3): ((0,), 1.0)}